<a href="https://colab.research.google.com/github/rybott/Forensic_Operation_Plan_Response/blob/main/test_notebook_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the clean version of Test_Work.ipynb
It contains the same code, but with only the core components of the generator

## Modules Used

In [192]:
import pandas as pd
import numpy as np
from datetime import timedelta
import random
import duckdb as ddb

# Further Analysis
import matplotlib.pyplot as plt
# import plotly.express as px

random.seed(0)

## Sales Invoices

In [193]:
def generate_sales_data(num_records, start_date, end_date, regions_df, products_df):
    # Lists to store generated data
    order_dates = []
    shipment_dates = []
    regions = []
    countries = []
    product_types = []
    unit_prices = []
    unit_costs = []
    quantities = []

    # Generate data for each record
    for _ in range(num_records):

        region_row = regions_df.sample().iloc[0]
        regions.append(region_row['StoreID'])
        countries.append(region_row['StoreName'])

        # Randomly select product and corresponding unit price and cost
        product_row = products_df.sample().iloc[0]
        product_types.append(product_row['Product'])
        unit_prices.append(product_row['Unit_Price'])
        unit_costs.append(product_row['Unit_Cost'])

        # Random order date between start and end date
        random_date = start_date + timedelta(days=np.random.randint(0, (end_date-start_date).days))
        order_dates.append(random_date)
        # Ship date between 1 and 50 days after order date
        shipment_dates.append(random_date + timedelta(days=np.random.randint(1, 51)))

        # Random quantity between 1000 and 10000
        quantities.append(np.random.randint(1000, 10001))


    sales_df = pd.DataFrame({
        'Order_Date': order_dates,
        'Shipment_Date': shipment_dates,
        'StoreID': regions,
        'StoreName': countries,
        'Product_Type': product_types,
        'Unit_Price': unit_prices,
        'Unit_Cost': unit_costs,
        'Quantity': quantities,
        'Revenue': np.array(unit_prices) * np.array(quantities),
        'Total_Cost': np.array(unit_costs) * np.array(quantities)
    })

    sales_df['Total Profit'] = sales_df['Revenue'] - sales_df['Total_Cost']

    return sales_df


regions_data = {
    'StoreID': [1,2,3,4,5,6,7,8,9,10],
    'StoreName': ["StoreNY","StoreLA","StorePA","StoreKY","StoreWA","StoreDC","StoreFL","StoreAL","StoreTX","StoreMN",]
}
regions_df = pd.DataFrame(regions_data)

products_data = {
    'Product': ["Office Supplies","Vegetables","Fruits","Cosmetics","Cereal","Baby Food","Beverages","Snacks","Clothes","Household","Personal Care","Meat"],
    'Unit_Price': [651.21,154.06,9.33,437.2,205.7,255.28,47.45,152.58,109.28,668.27,81.73,421.89],
    'Unit_Cost': [524.96,90.93,6.92,263.33,117.11,159.42,31.79,97.44,35.84,502.54,56.67,364.69]
}

products_df = pd.DataFrame(products_data)

start_date = pd.Timestamp('2015-01-01')
Start_Year = start_date.strftime('%Y')
end_date = pd.Timestamp('2018-12-31')
End_Year = int(end_date.strftime('%Y'))
num_records = 1000

sales_df = generate_sales_data(num_records, start_date, end_date, regions_df, products_df)

print(sales_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order_Date     1000 non-null   datetime64[ns]
 1   Shipment_Date  1000 non-null   datetime64[ns]
 2   StoreID        1000 non-null   int64         
 3   StoreName      1000 non-null   object        
 4   Product_Type   1000 non-null   object        
 5   Unit_Price     1000 non-null   float64       
 6   Unit_Cost      1000 non-null   float64       
 7   Quantity       1000 non-null   int64         
 8   Revenue        1000 non-null   float64       
 9   Total_Cost     1000 non-null   float64       
 10  Total Profit   1000 non-null   float64       
dtypes: datetime64[ns](2), float64(5), int64(2), object(2)
memory usage: 86.1+ KB
None


## Purchase Orders

In [194]:
sales_data = sales_df
sales_data = sales_data.sort_values(by=['Order_Date'])



product_types = ['Cereal', 'Snacks', 'Beverages', 'Baby Food', 'Meat', 'Fruits', 'Vegetables', 'Personal Care', 'Cosmetics', 'Household', 'Office Supplies', 'Clothes']


purchase_orders = pd.DataFrame(columns=['OrderID', 'Product', 'Vendor', 'Quantity', 'OrderDate'])
Inventory_dict = {'Cereal':0, 'Snacks':0, 'Beverages':0, 'Baby Food':0, 'Meat':0, 'Fruits':0, 'Vegetables':0, 'Personal Care':0, 'Cosmetics':0, 'Household':0, 'Office Supplies':0, 'Clothes':0}
Vendor_dict = {'Cereal':'Foodco', 'Snacks':'Foodco', 'Beverages':'Foodco', 'Baby Food':'Foodco', 'Meat':'Farmco', 'Fruits':'Farmco', 'Vegetables':'Farmco', 'Personal Care':'Beautyco', 'Cosmetics':'Beautyco', 'Household':'Homeco', 'Office Supplies':'Homeco', 'Clothes':'Fashionco'}


Product_Ordered = []
Quantity_Purchased = []
Date_Purchased = []
InventoryBeg = []
InventoryEnd = []
Vendor = []
Quantity_Sold = []


# purchase orders

for index, row in sales_data.iterrows():
  x = 0
  product = row['Product_Type']
  quantity_sold = row['Quantity']
  order_date = row['Order_Date']
  Unit_Cost = row['Unit_Cost']

  if product in Inventory_dict.keys():

    '''
    These aditions take the beginning inventory of a product on the date of sale, decreases
    from sold quantities, and increases from POs and then calc COGS

    You can also insert here a new table of externalities that effect COGS or price delta logic
    '''
    InventoryBeg.append(Inventory_dict[product])
    Quantity_Sold.append(quantity_sold * -1)

    if quantity_sold > Inventory_dict[product]:
      purchase_quantity = (quantity_sold - Inventory_dict[product]) * 1.05
      Inventory_dict[product] = (purchase_quantity + Inventory_dict[product]) - quantity_sold
      Product_Ordered.append(product)
      Quantity_Purchased.append(purchase_quantity)
      Date_Purchased.append(order_date)
      Ven = Vendor_dict[product]
      Vendor.append(Ven)
    else:
      Inventory_dict[product] = Inventory_dict[product] - quantity_sold

    InventoryEnd.append(Inventory_dict[product])



PoInv_df = pd.DataFrame(
    {'Order_Date': Date_Purchased,
     'Vendor': Vendor,
     'Quantity': Quantity_Purchased,
     'Product': Product_Ordered,
     'InventoryBeg': InventoryBeg,
     'InventoryEnd': InventoryEnd,
     'Quantity)Sold': Quantity_Sold,
    })

Qry_PO ='''
    SELECT Vendor,
    Product,
    SUM(Quantity) AS Quantity,
    YEAR(Order_Date) AS Year,
    QUARTER(Order_Date) AS Quarter,
    CAST(
      CASE
        WHEN QUARTER(Order_Date) = 1
          THEN CONCAT(YEAR(Order_Date), '-01-01')
        WHEN QUARTER(Order_Date) = 2
          THEN CONCAT(YEAR(Order_Date), '-04-01')
        WHEN QUARTER(Order_Date) = 3
          THEN CONCAT(YEAR(Order_Date), '-07-01')
        WHEN QUARTER(Order_Date) = 4
          THEN CONCAT(YEAR(Order_Date), '-10-01')
      END AS DATE) AS Purchase_Date
    FROM PoInv_df
    GROUP BY Vendor, Product, YEAR(Order_Date), QUARTER(Order_Date)
    ORDER BY Year, Quarter
'''

df_PO = ddb.sql(Qry_PO).df()

df_PO = df_PO.merge(products_df[['Product','Unit_Cost']], on ='Product', how = 'left')

print(df_PO.info())



<class 'pandas.core.frame.DataFrame'>
Int64Index: 190 entries, 0 to 189
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Vendor         190 non-null    object        
 1   Product        190 non-null    object        
 2   Quantity       190 non-null    float64       
 3   Year           190 non-null    int64         
 4   Quarter        190 non-null    int64         
 5   Purchase_Date  190 non-null    datetime64[ns]
 6   Unit_Cost      190 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 11.9+ KB
None


## Inventory

In [195]:
Inv_DEC = pd.DataFrame({"Date" : sales_df['Shipment_Date'],"Product":sales_df["Product_Type"],"Quantity":sales_df["Quantity"],"Unit_Cost":sales_df["Unit_Cost"]*-1})
Inv_INC = pd.DataFrame({"Date": df_PO['Purchase_Date'],"Product":df_PO['Product'],"Quantity":df_PO['Quantity'],"Unit_Cost":df_PO["Unit_Cost"],"Vendor":df_PO['Vendor']})

df_INV = Inv_DEC.append(Inv_INC,ignore_index=True)

print(df_INV.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190 entries, 0 to 1189
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       1190 non-null   datetime64[ns]
 1   Product    1190 non-null   object        
 2   Quantity   1190 non-null   float64       
 3   Unit_Cost  1190 non-null   float64       
 4   Vendor     190 non-null    object        
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 46.6+ KB
None


<ipython-input-195-cee86e0060dd>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_INV = Inv_DEC.append(Inv_INC,ignore_index=True)


## Total Revenue Calculations

In [196]:
# Total Revenue
qry_Trev = '''
    SELECT YEAR(Shipment_Date) AS Year, SUM(Revenue) AS Revenue
    FROM sales_df
    GROUP BY Year
    Order By Year
'''

# Total Revenue Per Product Per year
qry_rev = '''
    SELECT YEAR(Shipment_Date) AS Year, Product_Type AS Product, SUM(Revenue) AS Revenue, Unit_Cost
    FROM sales_df
    GROUP BY Year, Product, Unit_Cost
    Order By Year
'''

df_Trev = ddb.sql(qry_Trev).df()
df_rev = ddb.sql(qry_rev).df()

# Rev_dict = dict(zip(df_Trev['Year'],zip(df_Trev['Revenue'],df_Trev['Unit_Cost']))
Rev_dict = dict(zip(df_Trev['Year'],df_Trev['Revenue']))

## Scenario 1

### COGS

In [197]:
# COGS Quantities = Beg Inventory + Purchases - End Inventory
PoInv_df['COGS_Quantity'] =  PoInv_df['InventoryBeg'] + PoInv_df['Quantity'] - PoInv_df['InventoryEnd']

# Add the unit costs (This could be where you add more sophisticated logic for price changes)
PoInv_df = PoInv_df.merge(products_df[['Product','Unit_Cost']], on ='Product', how = 'left')

COGS_df1 = PoInv_df[['Order_Date','Product','COGS_Quantity','Unit_Cost']]

COGS_qry1 = '''
    SELECT
      CAST(
        CASE
          WHEN Quarter(Order_Date) = 1
            THEN CONCAT(YEAR(Order_Date), '-01-30')
          WHEN Quarter(Order_Date) = 2
            THEN CONCAT(YEAR(Order_Date), '-04-30')
          WHEN Quarter(Order_Date) = 3
            THEN CONCAT(YEAR(Order_Date), '-07-30')
          WHEN Quarter(Order_Date) = 4
            THEN CONCAT(YEAR(Order_Date), '-10-30')
        END
        AS DATE) AS Date,
      Product,
      SUM((COGS_Quantity * Unit_Cost)) AS Value
    FROM COGS_df1
    GROUP BY Date, Product
  ORDER BY Date
'''
COGS_df_byProduct = ddb.sql(COGS_qry1).df()

COGS_qry2 = '''
    SELECT Date, SUM(Value) AS Value
    FROM COGS_df_byProduct
    GROUP BY Date
    ORDER BY Date
'''

COGS_df = ddb.sql(COGS_qry2).df()

# Gross Margin
#   Yearly Rev - Yearly COGS
# Yearly COGS

Yr_COGS_qry = '''
    SELECT
        YEAR(Date) AS Year,
        Sum(Value) AS Value
    FROM COGS_df
    GROUP BY Year
'''
COGSyr_df = ddb.sql(Yr_COGS_qry).df()
COGS_dict = dict(zip(COGSyr_df['Year'],COGSyr_df['Value']))

PRC_dict = {}
PM_dict = {}
# Substract COGS from Rev each year
for Year in Rev_dict.keys():
  try:
    PRC_dict[Year] = {"Rev":Rev_dict[Year],"COGS":COGS_dict[Year],"PM":Rev_dict[Year]-COGS_dict[Year]}
    PM_dict[Year] = Rev_dict[Year]-COGS_dict[Year]
  except:
    PRC_dict[Year] = {"Rev":Rev_dict[Year],"COGS":0,"PM":0}
    PM_dict[Year] = 0
PM_Rev_df = pd.DataFrame({'PM':PM_dict,'REV':Rev_dict})

print(PM_Rev_df)

                PM           REV
2015  8.404917e+07  3.671214e+08
2016  1.042148e+08  3.394847e+08
2017  1.094356e+08  3.534040e+08
2018  9.763516e+07  3.552504e+08
2019  0.000000e+00  2.151959e+07


### Expenses for Scenario 1

In [198]:
Expenses = {
    'Avg_NetProfit':(3,"Not used to calc net profit"),
    'Rent': (6,10),
    'Insurance': (2,4),
    'Wages': (15,20),
    'Ads': (6,8),
    'Taxes': (13, "A fixed amount")
}

Exp_dict = {}
Exp_list = []

for i, year in enumerate(PRC_dict.keys()):
    gross = PRC_dict[year]["PM"]
    Rev = PRC_dict[year]["Rev"]
    COGS = PRC_dict[year]["COGS"]
    Year = year


    Rent = (random.randrange(Expenses['Rent'][0],Expenses['Rent'][1])/100)*gross
    Insurance = (random.randrange(Expenses['Insurance'][0],Expenses['Insurance'][1])/100)*gross
    Wages = (random.randrange(Expenses['Wages'][0],Expenses['Wages'][1])/100)*gross
    Ads = (random.randrange(Expenses['Wages'][0],Expenses['Wages'][1])/100)*gross
    Net = gross - (Rent+Insurance+Wages+Ads)
    EBIT = (1 - (Expenses['Taxes'][0]/100)) * Net
    Tax = (Expenses['Taxes'][0]/100) * Net
    Net_In = EBIT - Tax

    Exp_dict = {'Year':Year,'Rev':Rev,'COGS':COGS,'Gross_Profit':gross,'Rent':Rent,'Insurance':Insurance,'Wages':Wages,'Ads':Ads,'Net_Profit':Net,'EBIT': EBIT, 'Taxes':Tax, 'Net_Income': Net_In}
    Exp_list.append(Exp_dict)


Exp_df = pd.DataFrame(Exp_list)

print(Exp_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Year          5 non-null      int64  
 1   Rev           5 non-null      float64
 2   COGS          5 non-null      float64
 3   Gross_Profit  5 non-null      float64
 4   Rent          5 non-null      float64
 5   Insurance     5 non-null      float64
 6   Wages         5 non-null      float64
 7   Ads           5 non-null      float64
 8   Net_Profit    5 non-null      float64
 9   EBIT          5 non-null      float64
 10  Taxes         5 non-null      float64
 11  Net_Income    5 non-null      float64
dtypes: float64(11), int64(1)
memory usage: 608.0 bytes
None


In [199]:
# Individual Expenses

num_years = len(Exp_df['Year'])
num_months = num_years*12

rent = Exp_df['Rent'].sum() / num_months
Insurance = Exp_df['Insurance'].sum() / num_months
Wages = Exp_df['Wages'].sum() / num_months

end_year_str = str(end_date)[:4]

# Struct of List - Year, Date, Amount, Description
Comp_Exp_List = []


for year in Exp_df['Year']:
  if int(year) <= int(end_year_str):

    # Rent
    for month in range(12):
      Year = year
      Date = f"{Year}-1-{month+1}"
      Amount = rent
      Description = "Rent"
      sub_dict = {'Year':Year, 'Date':Date, 'Amount':Amount, 'Description':Description}
      Comp_Exp_List.append(sub_dict)

    # Insurance
    for month in range(12):
      Year = year
      Date = f"{Year}-1-{month+1}"
      Amount = Insurance
      Description = "Insurance"
      sub_dict = {'Year':Year, 'Date':Date, 'Amount':Amount, 'Description':Description}
      Comp_Exp_List.append(sub_dict)

    # Wages
    for month in range(12):
      Year = year
      Date = f"{Year}-1-{month+1}" # in the future it can be twice a month for 2 week pay
      Amount = Wages
      Description = "Wages"
      sub_dict = {'Year':Year, 'Date':Date, 'Amount':Amount, 'Description':Description}
      Comp_Exp_List.append(sub_dict)

    # Ads
    # Determine how many expenses per year
    No_Ads = random.randrange(3,12)
    Ad_yrly = Exp_df[Exp_df['Year'] == year]['Ads'].iloc[0]
    amounts = []
    adj_amounts = []
    days = []
    months = []

    for ad in range(No_Ads):
      Day = random.randrange(1,28)
      month = random.randrange(1,12)
      amount = random.randint(1,100)
      amounts.append(amount)
      days.append(Day)
      months.append(month)

    amount_total = sum(amounts)

    for amount in amounts:
      amount = (amount/amount_total)*Ad_yrly
      adj_amounts.append(amount)

    for month, day, amount in zip(months,days,adj_amounts):
      Year = year
      Date =f"{Year}-1-{month}"
      Amount = amount
      Description = "Ads"
      sub_dict = {'Year':Year, 'Date':Date, 'Amount':Amount, 'Description':Description}
      Comp_Exp_List.append(sub_dict)
  else:
    pass

Comp_Exp_df = pd.DataFrame(Comp_Exp_List)
print(Comp_Exp_df['Date'])
Comp_Exp_df['Date'] = pd.to_datetime(Comp_Exp_df['Date'],format='%Y-%m-%d')
print(Comp_Exp_df.info())

0       2015-1-1
1       2015-1-2
2       2015-1-3
3       2015-1-4
4       2015-1-5
         ...    
168    2018-1-11
169     2018-1-2
170     2018-1-3
171     2018-1-2
172     2018-1-9
Name: Date, Length: 173, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Year         173 non-null    int64         
 1   Date         173 non-null    datetime64[ns]
 2   Amount       173 non-null    float64       
 3   Description  173 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 5.5+ KB
None


# Financial Reporting

## Journal Entries

In [200]:
# Create the Journal Entries Dataframe

Account = []
Date = []
Description = []
Amount = []
Dr_Cr = []

Gen_Journal_df = pd.DataFrame({
    'Account':Account,
    'Date':Date,
    'Description': Description,
    'Amount':Amount,
    'Dr_Cr':Dr_Cr})

In [201]:
# Sales Journal

Sales_data = sales_df.copy()

Accounts = []
Dates = []
Descriptions = []
Amounts = []
Dr_Crs = []
ID = 0


for index, row in Sales_data.iterrows():
  ID = ID + 1
  date = row['Shipment_Date']
  amount = row['Revenue']
  desc = f"{ID}_Sale of Inventory "
  COGS = row['Total_Cost']
  Cash_acc = 1000
  AR_acc = 1010
  Rev_acc = 4000
  COGS_acc = 5000
  INV_acc = 1100

  rand_acc = random.randint(1,100)
  if rand_acc < 70:
    Sale_acc = AR_acc
  else:
    Sale_acc = Cash_acc

  # Sale
  Accounts.append(Sale_acc)
  Dates.append(date)
  Descriptions.append(desc)
  Amounts.append(amount)
  Dr_Cr = "Debit"
  Dr_Crs.append(Dr_Cr)

  Accounts.append(Rev_acc)
  Dates.append(date)
  Descriptions.append(desc)
  Amounts.append(amount)
  Dr_Cr = "Credit"
  Dr_Crs.append(Dr_Cr)


Sales_Journal_df = pd.DataFrame({
    'Account':Accounts,
    'Date':Dates,
    'Description': Descriptions,
    'Amount':Amounts,
    'Dr_Cr':Dr_Crs})

Gen_Journal_df = pd.concat([Gen_Journal_df,Sales_Journal_df], ignore_index=True)

In [202]:
print(Sales_Journal_df.info())
print(Gen_Journal_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Account      2000 non-null   int64         
 1   Date         2000 non-null   datetime64[ns]
 2   Description  2000 non-null   object        
 3   Amount       2000 non-null   float64       
 4   Dr_Cr        2000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 78.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Account      2000 non-null   float64       
 1   Date         2000 non-null   datetime64[ns]
 2   Description  2000 non-null   object        
 3   Amount       2000 non-null   float64       
 4   Dr_Cr        2000 non-null   object        
dtypes: datetim

In [203]:
# Purchase Order

PO_data = df_PO.copy()

Accounts = []
Dates = []
Descriptions = []
Amounts = []
Dr_Crs = []
ID = 0


for index, row in PO_data.iterrows():
  ID = ID + 1
  date = row['Purchase_Date']
  amount = row['Unit_Cost']*row['Quantity']
  desc = f"{ID}_Purchase of Inventory "
  AP_acc = 2000
  INV_acc = 1100

  Accounts.append(INV_acc)
  Dates.append(date)
  Descriptions.append(desc)
  Amounts.append(amount)
  Dr_Cr = "Debit"
  Dr_Crs.append(Dr_Cr)

  Accounts.append(AP_acc)
  Dates.append(date)
  Descriptions.append(desc)
  Amounts.append(amount)
  Dr_Cr = "Credit"
  Dr_Crs.append(Dr_Cr)


Inventory_Journal_df = pd.DataFrame({
    'Account':Accounts,
    'Date':Dates,
    'Description': Descriptions,
    'Amount':Amounts,
    'Dr_Cr':Dr_Crs})

Gen_Journal_df = pd.concat([Gen_Journal_df,Inventory_Journal_df,], ignore_index=True)

In [204]:
print(Inventory_Journal_df.info())
print(Gen_Journal_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Account      380 non-null    int64         
 1   Date         380 non-null    datetime64[ns]
 2   Description  380 non-null    object        
 3   Amount       380 non-null    float64       
 4   Dr_Cr        380 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 15.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Account      2380 non-null   float64       
 1   Date         2380 non-null   datetime64[ns]
 2   Description  2380 non-null   object        
 3   Amount       2380 non-null   float64       
 4   Dr_Cr        2380 non-null   object        
dtypes: datetime6

In [205]:
# COGS

COGS_data = COGS_df.copy()

Accounts = []
Dates = []
Descriptions = []
Amounts = []
Dr_Crs = []
ID = 0

for index, row in COGS_data.iterrows():
  ID = ID + 1

  date = row['Date']
  amount = row['Value']
  desc = f"{ID}_Quarterly COGS"
  COGS = 5000
  INV_acc = 1100

  Accounts.append(COGS)
  Dates.append(date)
  Descriptions.append(desc)
  Amounts.append(amount)
  Dr_Cr = "Debit"
  Dr_Crs.append(Dr_Cr)

  Accounts.append(INV_acc)
  Dates.append(date)
  Descriptions.append(desc)
  Amounts.append(amount)
  Dr_Cr = "Credit"
  Dr_Crs.append(Dr_Cr)


COGS_Journal_df = pd.DataFrame({
    'Account':Accounts,
    'Date':Dates,
    'Description': Descriptions,
    'Amount':Amounts,
    'Dr_Cr':Dr_Crs})

Gen_Journal_df = pd.concat([Gen_Journal_df,COGS_Journal_df,], ignore_index=True)

In [206]:
# Expenses

exp_data = Comp_Exp_df.copy()

Accounts = []
Dates = []
Descriptions = []
Amounts = []
Dr_Crs = []
ID = 0

for index, row in exp_data.iterrows():
  ID = ID + 1
  date = row['Date']
  amount = row['Amount']
  exp = row['Description']
  desc = f"{ID}_{exp} Expense"
  AP_acc = 2000
  Cash_acc = 1000

  match exp:
    # Fixed Cost Prepaid Exps are being accrued
    case 'Rent':
      account = 5110
      pay_acc = Cash_acc

    case 'Wages':
      account = 5010
      pay_acc = Cash_acc

    case 'Insurance':
      account = 5050
      pay_acc = Cash_acc

    case 'Ads':
      account = 5020
      rand_acc = random.randint(1,100)
      if rand_acc < 70:
        pay_acc = AP_acc
      else:
        pay_acc = Cash_acc

  Accounts.append(account)
  Dates.append(date)
  Descriptions.append(desc)
  Amounts.append(amount)
  Dr_Cr = "Debit"
  Dr_Crs.append(Dr_Cr)

  Accounts.append(pay_acc)
  Dates.append(date)
  Descriptions.append(desc)
  Amounts.append(amount)
  Dr_Cr = "Credit"
  Dr_Crs.append(Dr_Cr)

Exp_Journal_df = pd.DataFrame({
  'Account':Accounts,
  'Date':Dates,
  'Description': Descriptions,
  'Amount':Amounts,
  'Dr_Cr':Dr_Crs})

Gen_Journal_df = pd.concat([Gen_Journal_df,Exp_Journal_df,], ignore_index=True)


In [207]:
print(Exp_Journal_df.info())
print(Gen_Journal_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Account      346 non-null    int64         
 1   Date         346 non-null    datetime64[ns]
 2   Description  346 non-null    object        
 3   Amount       346 non-null    float64       
 4   Dr_Cr        346 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 13.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2758 entries, 0 to 2757
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Account      2758 non-null   float64       
 1   Date         2758 non-null   datetime64[ns]
 2   Description  2758 non-null   object        
 3   Amount       2758 non-null   float64       
 4   Dr_Cr        2758 non-null   object        
dtypes: datetime6

In [208]:
Gen_Journal_df['Date'] = pd.to_datetime(Gen_Journal_df['Date'],format='%d-%m-%Y')

In [209]:
# Pay off AP and Recieve Recievables

Accounts = []
Dates = []
Descriptions = []
Amounts = []
Dr_Crs = []

for index, row in Gen_Journal_df.iterrows():
    # AP = 2000
    if row['Account'] == 2000:
        Purchase_date = row['Date']
        # All AP is Net 30 terms and we way in 30 days
        Cash_Paid_Date = row['Date'] + timedelta(days=30)
        Cash_Acc = 1000
        AP_Acc = 2000
        desc = "Paided down AP liability"
        amount = row["Amount"]

        Accounts.append(AP_Acc)
        Dates.append(Cash_Paid_Date)
        Descriptions.append(desc)
        Amounts.append(amount)
        Dr_Cr = "Debit"
        Dr_Crs.append(Dr_Cr)

        Accounts.append(Cash_Acc)
        Dates.append(Cash_Paid_Date)
        Descriptions.append(desc)
        Amounts.append(amount)
        Dr_Cr = "Credit"
        Dr_Crs.append(Dr_Cr)

    # AR = 1010
    elif row['Account'] == 1010:
        Purchase_date = row['Date']
        # All AP is Net 30 terms and we way in 30 days
        Days_Since_Paid = random.randint(5,100)
        Cash_Paid_Date = row['Date'] + timedelta(days=Days_Since_Paid)
        Cash_Acc = 1000
        AR_Acc = 1010
        desc = f"Recieved Recievables after {Days_Since_Paid} late"
        amount = row["Amount"]

        Accounts.append(Cash_Acc)
        Dates.append(Cash_Paid_Date)
        Descriptions.append(desc)
        Amounts.append(amount)
        Dr_Cr = "Debit"
        Dr_Crs.append(Dr_Cr)

        Accounts.append(AR_Acc)
        Dates.append(Cash_Paid_Date)
        Descriptions.append(desc)
        Amounts.append(amount)
        Dr_Cr = "Credit"
        Dr_Crs.append(Dr_Cr)

    else:
        pass

AP_AR_Journal_df = pd.DataFrame({
'Account':Accounts,
'Date':Dates,
'Description': Descriptions,
'Amount':Amounts,
'Dr_Cr':Dr_Crs})

Gen_Journal_df = pd.concat([Gen_Journal_df,AP_AR_Journal_df,], ignore_index=True)


In [210]:
print(AP_AR_Journal_df.info())
print(Gen_Journal_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Account      1830 non-null   int64         
 1   Date         1830 non-null   datetime64[ns]
 2   Description  1830 non-null   object        
 3   Amount       1830 non-null   float64       
 4   Dr_Cr        1830 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 71.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4588 entries, 0 to 4587
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Account      4588 non-null   float64       
 1   Date         4588 non-null   datetime64[ns]
 2   Description  4588 non-null   object        
 3   Amount       4588 non-null   float64       
 4   Dr_Cr        4588 non-null   object        
dtypes: datetim

# Calculating T-balances for Each account and then by month
### Multi-Step Query
1. Select * but if Debit then Amount and if Credit then Amount(-1)
2. Select * group by account title which will give you the balance of each account totals
3. Make a new adjusted T-balance for each of the years seperated by month

In [211]:
# Step 1
qry_Tacc1 = '''
  SELECT
    Account,
    Date,
    CASE
        WHEN Dr_Cr = 'Credit'
          THEN Amount * -1
        WHEN Dr_Cr = 'Debit'
          THEN Amount
    END AS Amount
  FROM Gen_Journal_df
'''
Tacc1 = ddb.sql(qry_Tacc1).df()


# Step 2
qry_Tacc2 = '''
  SELECT
    Account,
    YEAR(Date) AS Year,
    ROUND(SUM(Amount),2) AS Amount
  FROM Tacc1
  GROUP BY Account, Year
  ORDER BY Year, Account
'''
Tacc2 = ddb.sql(qry_Tacc2).df()

print(Tacc2.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Account  43 non-null     float64
 1   Year     43 non-null     int64  
 2   Amount   43 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 1.1 KB
None


In [212]:
with pd.ExcelWriter('Tacc_pre.xlsx') as writer:
    Tacc2.to_excel(writer, sheet_name='Tacc2', index=False)


# Other Balance Sheet Transactions
## Increase in Assets will also increase:
1. Loans (Notes Payable)
2. Common Shares
    - Useful for injecting cash if needed
    - may need to pay dividends
3. Retained Earnings

## Adding Assets:
1. Land (Not in this scenario they rent)
2. Equipment
3. Buildings
    - Depreciation for Equipment and Buildings

## Implementation
Either the user can define everything, user define equipment amount and computer will determine depreciation, or the program will decided what to do based on retained earnings and the following criteria
1. Equipment will depreciate at 5% of retained earnings over 20 years for scenario one, Equipment value will be that number * 20 years (straight line deprecation) but because EBITA will be different every year I will take the average of all the years for that calc.
2. Buildings will depreciate at 10% of retained earnings over 30 years, same calculations
3. The amount of depr before the start of the recorded period will have to be recorded.

# Steps for Incorporation of Other Assets
1. Skip creating the Financial Statments until you have the entire general ledger
2. Use Tacc df to find the retained earnings (Which this is based on)
3. Then just use the same code from yesterday to create the general journal and a second Tacc which is much less memory
4. Use the same code just with the full statment to generate the statments and pray that they are balanced.

## Other Things I have to add directly to general journal one time - added to year 1
1. Equipment Book Value
2. Building Book Value
3. Current Accum Depreciation for BOTH
4. Notes Payable CURRENT VALUE (Note total - payed already)

In [213]:
# Calculating Retained Earnings
Tacc2_filtered = Tacc2[Tacc2['Year'] <= End_Year]
def calculate_ebita(row):
    if row['Account'] == 4000 or row['Account'] == 5000:
        return row['Amount']
    elif row['Account'] in [5110, 5050, 5010, 5020]:
        return row['Amount']
    else:
        return 0
Tacc2_filtered['EBITA'] = Tacc2_filtered.apply(calculate_ebita, axis=1)

unadjusted_retained_earnings = Tacc2_filtered.groupby('Year')['EBITA'].sum().reset_index()
print(unadjusted_retained_earnings.info())
print(unadjusted_retained_earnings)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Year    4 non-null      int64  
 1   EBITA   4 non-null      float64
dtypes: float64(1), int64(1)
memory usage: 192.0 bytes
None
   Year        EBITA
0  2015 -47948667.98
1  2016 -63644006.15
2  2017 -70113805.04
3  2018 -57272341.78


<ipython-input-213-9a5114150cb3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tacc2_filtered['EBITA'] = Tacc2_filtered.apply(calculate_ebita, axis=1)


In [214]:
# Calculating Depreciation
Test_EBITA = []
Equipment_depr = []
Building_depr = []
Years_list = []
Unadjusted_RetainedEarnings = []

for year in unadjusted_retained_earnings['Year']:
  # Unadjusted Retained Earnings = EBITA || Retained Earnings = Net Income calced now
  EBITA = unadjusted_retained_earnings.loc[(unadjusted_retained_earnings['Year'] == year), 'EBITA'].sum()
  Test_EBITA.append(EBITA)
  equipment_depr = EBITA * .05
  Equipment_depr.append(equipment_depr)
  building_depr = EBITA *.1
  Building_depr.append(building_depr)
  Years_list.append(year)
  Unadjusted_RetainedEarnings.append(EBITA)

Yearly_Equipment_depr = sum(Equipment_depr) / len(Equipment_depr) *-1
Yearly_Building_depr = sum(Building_depr) / len(Building_depr) *-1

'''
print("EBITAS: ",Test_EBITA)
print("Yearly Depr: ", Yearly_Equipment_depr)
print("EBITAS: ",Test_EBITA)
'''

# 20 and 30 Year useful life respectfully
Value_Equipment_New = Yearly_Equipment_depr * 20
Value_Building_New = Yearly_Building_depr * 30

Accounts = []
Dates = []
Descriptions = []
Amounts = []
Dr_Crs = []
RetainedEarnings = []

# Scenario 1 - All assets will be 10 years into their book values
# This variables will be the counters for depreciation
Value_Equipment = Value_Equipment_New*.5 # Half useful life
Value_Equipment_depr = Value_Equipment_New*(10/20) # Half Depreciated

Value_Building = Value_Building_New*(20/30)
Value_Building_depr = Value_Building_New*(10/30)

# Notes will be fixed 5% interest for 20 years for equipment
Original_Notes = Value_Equipment_New
# I can seperate them into monthly payments another day

Note_Payments_Yearly = Original_Notes / 20 # Years of notes
Interest_Payments_Yearly = Original_Notes * .05


Notes_Remaining = Original_Notes *.5 # 10 years of payments already done
Notes_Counter = Notes_Remaining

print("Depreciation")
print("EBITAS: ",Test_EBITA)
print("Yearly Depr: ", Yearly_Equipment_depr)
print("Value of Equipment NEW: ",Value_Equipment_New)
print("Value of Equipment BK: ",Value_Equipment)
print("Value of Equipment Depr: ",Value_Equipment_depr)
print("Notes")
print("Note_New: ",Original_Notes)
print("Note_remaining: ",Notes_Remaining)
print("Note yearly payment: ",Note_Payments_Yearly)
print("Note interest payment: ",Interest_Payments_Yearly)


# Here we are adding depreciation notes payable and retained earnings to the additional journal
for year in unadjusted_retained_earnings['Year']:
  if Value_Equipment != 0:
      Date = f"{year}-12-31"

    # Debit Depr Exp
      Accounts.append(5030)
      Dates.append(Date)
      Descriptions.append("Deprecation of Equipment")
      Amounts.append(Yearly_Equipment_depr)
      Dr_Cr = "Debit"
      Dr_Crs.append(Dr_Cr)

    # Credit depr - Equipment
      Accounts.append(1231)
      Dates.append(Date)
      Descriptions.append("Deprecation of Equipment")
      Amounts.append(Yearly_Equipment_depr)
      Dr_Cr = "Credit"
      Dr_Crs.append(Dr_Cr)

      Value_Equipment = Value_Equipment - Yearly_Equipment_depr
  else:
    pass

  if Notes_Counter != 0:
      Date = f"{year}-12-31"

    # Debit Notes Payable
      Accounts.append(2020)
      Dates.append(Date)
      Descriptions.append("Notes Payment")
      Amounts.append(Yearly_Equipment_depr*-1)
      Dr_Cr = "Debit"
      Dr_Crs.append(Dr_Cr)

    # Debit Interest Exp
      Accounts.append(5080)
      Dates.append(Date)
      Descriptions.append("Interest on Notes")
      Amounts.append(Interest_Payments_Yearly)
      Dr_Cr = "Debit"
      Dr_Crs.append(Dr_Cr)

    # Credit Cash
      Accounts.append(1000)
      Dates.append(Date)
      Descriptions.append("Notes/Interest Payment")
      Amounts.append(Interest_Payments_Yearly+Note_Payments_Yearly)
      Dr_Cr = "Credit"
      Dr_Crs.append(Dr_Cr)

      Notes_Counter = Notes_Counter - Note_Payments_Yearly
  else:
    pass

# Finally we add the inital values carried over for Notes Payable and Equipment

# Credit Notes Payable
Accounts.append(2020)
Dates.append(f"{Start_Year}-01-01")
Descriptions.append("Initial Note")
Amounts.append(Notes_Remaining *-1) # A removed -1
Dr_Cr = "Credit"
Dr_Crs.append(Dr_Cr)

# Credit depr - Equipment
Accounts.append(1231)
Dates.append(f"{Start_Year}-01-01")
Descriptions.append("Initial Accum Depr - Equip")
Amounts.append(Value_Equipment_depr) # B Added -1
Dr_Cr = "Credit"
Dr_Crs.append(Dr_Cr)

# Debit Equipment
Accounts.append(1230)
Dates.append(f"{Start_Year}-01-01")
Descriptions.append("Initial Equipment")
Amounts.append(Value_Equipment_New)
Dr_Cr = "Debit"
Dr_Crs.append(Dr_Cr)

# This is an inflow of Cash to make sure nothing is negative

# Debit Cash
Accounts.append(1000)
Dates.append(f"{Start_Year}-01-01")
Descriptions.append("Cash From Last Year")
Amounts.append(40000000)
Dr_Cr = "Debit"
Dr_Crs.append(Dr_Cr)

# Credit Retained Earnings
Accounts.append(3010)
Dates.append(f"{Start_Year}-01-01")
Descriptions.append("Cash From Last Year")
Amounts.append(40000000)
Dr_Cr = "Credit"
Dr_Crs.append(Dr_Cr)


Additional_Journal_df = pd.DataFrame({
'Account':Accounts,
'Date':Dates,
'Description': Descriptions,
'Amount':Amounts,
'Dr_Cr':Dr_Crs})

print("\n")
print(Gen_Journal_df.info())
print(Additional_Journal_df.info())
Gen_Journal_df2 = pd.concat([Gen_Journal_df,Additional_Journal_df], ignore_index=True)
print(Gen_Journal_df2.info())


Depreciation
EBITAS:  [-47948667.980000034, -63644006.15000002, -70113805.04000002, -57272341.779999994]
Yearly Depr:  2987235.2618750012
Value of Equipment NEW:  59744705.23750003
Value of Equipment BK:  29872352.618750013
Value of Equipment Depr:  29872352.618750013
Notes
Note_New:  59744705.23750003
Note_remaining:  29872352.618750013
Note yearly payment:  2987235.2618750012
Note interest payment:  2987235.2618750017


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4588 entries, 0 to 4587
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Account      4588 non-null   float64       
 1   Date         4588 non-null   datetime64[ns]
 2   Description  4588 non-null   object        
 3   Amount       4588 non-null   float64       
 4   Dr_Cr        4588 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 179.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeInde

In [215]:
# Using the Tacc now with all of the adjustments

# Step 1
qry_Tacc1_adjusted = '''
  SELECT
    Account,
    Date,
    CASE
        WHEN Dr_Cr = 'Credit'
          THEN Amount * -1
        WHEN Dr_Cr = 'Debit'
          THEN Amount
    END AS Amount
  FROM Gen_Journal_df2
'''
Tacc1_adjusted = ddb.sql(qry_Tacc1_adjusted).df()

# Step 2
qry_Tacc2_adjusted = '''
  SELECT
    Account,
    YEAR(CAST(Date AS DATE)) AS Year,  -- Casting the Date column to DATE
    ROUND(SUM(Amount),2) AS Amount
  FROM Tacc1_adjusted
  GROUP BY Account, YEAR(CAST(Date AS DATE))  -- Casting here as well for the GROUP BY clause
  ORDER BY Year, Account
'''
Tacc2_adjusted = ddb.sql(qry_Tacc2_adjusted).df()

print(Tacc2_adjusted.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Account  61 non-null     float64
 1   Year     61 non-null     int64  
 2   Amount   61 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 1.6 KB
None


In [216]:
# Income Statment
Year = []
Sales_Rev = []
COGS = []
Gross_Profit = []
Rent_exp = []
Insurance_exp = []
Salaries_exp = []
Ad_exp = []
Total_Operating_exp = []
EBITA = []
Interest_exp = []
Depreciation_exp = []
Income_Before_Tax = []
Income_Tax = []
Net_Income = []


for year in Tacc2_adjusted['Year'].unique():
  if year <= End_Year:
    Year.append(year)
    sales_rev = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==4000) & (Tacc2_adjusted['Year']==year),'Amount'].iloc[0] * -1
    Sales_Rev.append(sales_rev)
    cogs = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==5000) & (Tacc2_adjusted['Year']==year),'Amount'].iloc[0]
    COGS.append(cogs)
    gross_profit = sales_rev - cogs
    Gross_Profit.append(gross_profit)
    rent_exp = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==5110) & (Tacc2_adjusted['Year']==year),'Amount'].iloc[0]
    Rent_exp.append(rent_exp)
    insurance_exp = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==5050) & (Tacc2_adjusted['Year']==year),'Amount'].iloc[0]
    Insurance_exp.append(insurance_exp)
    salaries_exp = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==5010) & (Tacc2_adjusted['Year']==year),'Amount'].iloc[0]
    Salaries_exp.append(salaries_exp)
    ad_exp = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==5020) & (Tacc2_adjusted['Year']==year),'Amount'].iloc[0]
    Ad_exp.append(ad_exp)
    total_op_exp = salaries_exp + insurance_exp + rent_exp + ad_exp
    Total_Operating_exp.append(total_op_exp)
    EBITA.append(gross_profit - total_op_exp)
    interest_exp = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==5080) & (Tacc2_adjusted['Year']==year),'Amount'].iloc[0]
    Interest_exp.append(interest_exp)
    depr_exp = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==5030) & (Tacc2_adjusted['Year']==year),'Amount'].iloc[0]
    Depreciation_exp.append(depr_exp)
    ebt = (gross_profit - total_op_exp) - interest_exp - depr_exp
    Income_Before_Tax.append(ebt)
    net_income = ebt
    Net_Income.append(net_income)

Income_Statments = pd.DataFrame({
'Year':Year,
'Sales_Rev':Sales_Rev,
'COGS':COGS,
'Gross_Profit':Gross_Profit,
'Rent_exp':Rent_exp,
'Insurance_exp':Insurance_exp,
'Salaries_exp':Salaries_exp,
'Ad_exp':Ad_exp,
'Total_Operating_exp':Total_Operating_exp,
'EBITA':EBITA,
'Interest_exp':Interest_exp,
'Depreciation_exp':Depreciation_exp
'Income_Before_Tax':Income_Before_Tax,
'Net_Income':Net_Income})

In [217]:
Cash = []
ARec = []
Inventory = []
Total_Current_Assets = []
Land = []
Equpiment = []
Buildings = []
Net_Accum_Depreciation = []
Total_Assets = []
Accounts_Payable = []
Notes_Payable = []
Total_Liability = []
Retained_Earnings = []
Total_Equity = []
Total_Liabilities_Equity = []
Year = []
Balanced = []

for year in Tacc2_adjusted['Year'].unique():
    Year.append(year)
    cash = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==1000) & (Tacc2_adjusted['Year']==year),'Amount'].sum()
    Cash.append(cash)
    arec = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==1010) & (Tacc2_adjusted['Year']==year),'Amount'].sum()
    ARec.append(arec)
    inventory = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==1100) & (Tacc2_adjusted['Year']==year),'Amount'].sum()
    Inventory.append(inventory)
    tca = cash + arec + inventory
    Total_Current_Assets.append(tca)
    equipment = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==1230) & (Tacc2_adjusted['Year']==year),'Amount'].sum()
    Equpiment.append(equipment)
    equipment_depr = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==1231) & (Tacc2_adjusted['Year']==year),'Amount'].sum()
    Net_Accum_Depreciation.append(equipment_depr)
    total_asset = (cash + arec + inventory + equipment + equipment_depr).astype(float)
    Total_Assets.append(total_asset)
    ap = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==2000) & (Tacc2_adjusted['Year']==year),'Amount'].sum()
    Accounts_Payable.append(ap)
    notes = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==2020) & (Tacc2_adjusted['Year']==year),'Amount'].sum()
    Notes_Payable.append(notes)
    total_liability = ap + -notes
    Total_Liability.append(total_liability)
    retained_earnings = Tacc2_adjusted.loc[(Tacc2_adjusted['Account']==3010) & (Tacc2_adjusted['Year']==year),'Amount'].sum()
    Retained_Earnings.append(retained_earnings*-1)
    total_equity = retained_earnings
    Total_Equity.append(total_equity)
    total_liability_equity = (total_liability + total_equity).astype(float)
    Total_Liabilities_Equity.append(total_liability_equity)
    if total_liability_equity == total_asset:
        Balanced.append(True)
    else:
        Balanced.append(False)

Balance_Sheets_unadjusted = pd.DataFrame({
'Year':Year,
'Cash':Cash,
'Arec':ARec,
'Inventory':Inventory,
'Total_Current_Assets':Total_Current_Assets,
'Equipment':Equpiment,
'Net_Accum_depr':Net_Accum_Depreciation,
'Total_Assets':Total_Assets,
'Accounts_Payable':Accounts_Payable,
'Notes_Payable':Notes_Payable,
'Total_Liability': Total_Liability,
'Retained_Earnings':Retained_Earnings,
'Total_Equity':Total_Equity,
'Total_Liabilities_Equity':Total_Liabilities_Equity
})

Balance_Sheets = Balance_Sheets_unadjusted.cumsum().shift(1, fill_value=0) + Balance_Sheets_unadjusted
Balance_Sheets['Year'] = Year

### Cash Flow (After Adding other Balance Sheet Transactions)
|Code|**Cash Flow**|||
|---|---|---|---|
|0000|I. Operating Activities|||
|0015|Net Income||xxx|
|5030|Depreciation||xxx|
|1010|(+)/- Accounts Recivable||xxx|
|2000|+/(-) Accounts Payable||xxx|
|1100|(+)/- Inventory||xxx|
|0012|+/(-) Interest Income||xxx|
|0016||Net Cash From Operations|xxx|
|0000|II. Investing Activities|||
|0000|None For this Scenario|||
|0000|III. Financing Activities|||
|2020|(-)/+ Notes Payable||(xxx)|
|0018||Net Cash From Financing|xxx|
|0019|Begining Cash Balance||xxx|
|0020|End Cash Balance||xxx|
|0021|***Net Cash Flow***||xxx|

In [218]:
'''
%%shell
jupyter nbconvert --to html /content/Copy_of_Test_Work.ipynb
'''

'\n%%shell\njupyter nbconvert --to html /content/Copy_of_Test_Work.ipynb\n'

In [219]:

# Print Financial Statments
with pd.ExcelWriter('combined_workbook.xlsx') as writer:
    Gen_Journal_df2.to_excel(writer, sheet_name='Gen_Journal', index=False)
    Income_Statments.to_excel(writer, sheet_name='Income_Statements', index=False)
    Balance_Sheets_unadjusted.to_excel(writer, sheet_name='Balance_Sheets_Unadjusted', index=False)
    Balance_Sheets.to_excel(writer, sheet_name='Balance_Sheets_Adjusted', index=False)
    Tacc2_adjusted.to_excel(writer, sheet_name='Tacc2_Adjusted', index=False)

